## 예제 2.1 토큰화 코드

In [1]:
# 띄어쓰기 단위로 분리 원래는 서브워드 토큰화를 사용하는데 보여주기 식으로 일단 간단하게 구현함
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split() #이게 vocabulary가 되는거고
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## 예제 2.2 토큰 아이디에서 벡터로 변환

In [2]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
input_embeddings.shape
#1개의 문장에 5개의 토큰이 존재하고 16차원의 임베딩으로 변환한다.

torch.Size([1, 5, 16])

## 예제 2.3 절대적 위치 인코딩

In [4]:
embedding_dim = 16
max_position = 12
# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)#5,16
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)#12,16

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)#1,5
print(position_ids)
position_encodings = position_embed_layer(position_ids)#1 5 16
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

tensor([[0, 1, 2, 3, 4]])


torch.Size([1, 5, 16])

## 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [5]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

## 예제 2.5. 스케일 점곱 방식의 어텐션

In [6]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	weights = F.softmax(scores, dim=-1)
	return weights @ values

## 예제 2.6. 어텐션 연산의 입력과 출력

In [7]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


## 예제 2.7. 어텐션 연산을 수행하는 AttentionHead 클래스

In [8]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal#아마 나중에 마스크 쓸 때 대비해서 작성해둔듯?
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def compute_attention(self,querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return weights @ values

  def forward(self, querys, keys, values):
    outputs = self.compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 예제 2.8. 멀티 헤드 어텐션 구현

In [9]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def compute_attention(self,querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 4
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    # 여기서 @ 행렬 곱 연산에 대해 이해 안갔는데 chat gpt가 도와줬음
    # transpose로 마지막 두 차원 바꿔서 행렬 곱하는건 이해했는데 앞에 두 차원은 아예 제외 하길래 왜 그런가 했더니
    # @ 연산은 일단 무조건 마지막 두 차원만 행렬 곱 계산을 해준다고 함
    print(scores.shape)#1 4 5 5 각 token에 대한 attention score (5,4) @(4,5) 했으니 (5,5)나옴
    weights = F.softmax(scores, dim=-1)
    return weights @ values

  def forward(self, querys, keys, values):
    B, T, C = querys.size() # 배치, 토큰, 채널(임베딩)
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) #transpose를 통해 차원의위치 이동 (B, self.n_head, T, C // self.n_head)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    attention = self.compute_attention(querys, keys, values, self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)#contiguous()는 메모리 뭐 초기화 하는건데 잘 모르겠음
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 4, 5, 5])


torch.Size([1, 5, 16])

## 예제 2.9. 층 정규화 코드

In [10]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data


# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[-7.4506e-09, -6.5193e-09,  1.6764e-08,  0.0000e+00, -2.2352e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

## 예제 2.10. 피드 포워드 층 코드

In [11]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 예제 2.11. 인코더 층

In [12]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

## 예제 2.12. 인코더 구현

In [13]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])#인코더 몇개 쌓을지 만들어 줌

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output

## 예제 2.13. 디코더에서 어텐션 연산(마스크 어텐션)

In [14]:
def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1,n_head, 5, 5)
	if is_causal:# 이부분이 핵심! 마스크를 씌어주는거
		query_length = querys.size(2)#(batch,n_head,token,embedding)1,5,16
		key_length = keys.size(2)
		temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)#그럼 5x5 크기의 하삼각행렬이 생성될듯?
		scores = scores.masked_fill(temp_mask == False, float("-inf"))#마스크의 false 인부분은 -inf로 값을 바꿔버려라
	weights = F.softmax(scores, dim=-1) # (1,n_head, 5, 5)
	return weights @ values # (1, n_head, 5, 16)

In [20]:
temp_mask = torch.ones(5, 5, dtype=torch.bool).tril(diagonal=0)
print(temp_mask) #diagonal 인자는 대각 성분을 얼마나 올려줄지 정해줌 1이면 한칸 위로 올려서 대각 행렬 생성


tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])


## 예제 2.14. 크로스 어텐션이 포함된 디코더 층

In [15]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output, is_causal=is_causal))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

## 예제 2.15. 디코더 구현

In [16]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(tgt, src)
    return output